In [6]:
import pymysql
import pandas as pd

conn = pymysql.connect(
    host="localhost",
    user="root",
    password="ROOT",
    database="project_stream",
    port=3306
)

In [7]:
# create a function out of it

def connect_to_db():
    return pymysql.connect(
    host="localhost",
    user="root",
    password="ROOT",
    database="project_stream",
    port=3306)

In [8]:
db = connect_to_db()
cursor = db.cursor()

In [9]:
cursor

In [10]:
cursor.execute("SELECT COUNT(*) as total_suppliers FROM suppliers")

1

In [11]:
row = cursor.fetchone()

In [12]:
list(row)[0]

50

In [13]:
queries = {
    "Total Suppliers":"SELECT COUNT(*) as total_suppliers FROM suppliers",
    
    "Total products": "SELECT count(*) as total_products FROM products",
    
    "Total category" :"SELECT count(distinct category) as total_category FROM products",
    
    "Total sales value made in last 3 months":"""SELECT ROUND(SUM(ABS(se.change_quantity) * p.price), 2) AS total_sales_last_3_months
    FROM stock_entries se
    JOIN products p 
    ON p.product_id = se.product_id
    WHERE se.change_type = 'Sale'
    AND se.entry_date >= (
        SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH)
        FROM stock_entries
         )""",
    
    
    "total restock value made in last 3 months":"""SELECT ROUND(SUM(ABS(se.change_quantity) * p.price), 2) AS total_restock_last_3_months
    FROM stock_entries se
    JOIN products p 
      ON p.product_id = se.product_id
    WHERE se.change_type = 'Restock'
      AND se.entry_date >= (
            SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH)
            FROM stock_entries
         )""",
     
    "below reorder and no pending reorders":"""SELECT COUNT(*) FROM products as p
    WHERE p.stock_quantity < p.reorder_level
    AND p.product_id NOT IN (
    SELECT DISTINCT product_id FROM reorders WHERE status = 'pending'
    )"""	
    }

In [14]:
result = {}
for label,query in queries.items():
    cursor.execute(query)
    row = cursor.fetchone()
        
    result[label] = list(row)[0]

In [15]:
result

{'Total Suppliers': 50,
 'Total products': 200,
 'Total category': 5,
 'Total sales value made in last 3 months': 1529482.99,
 'total restock value made in last 3 months': 6691663.1,
 'below reorder and no pending reorders': 14}

In [16]:
def get_basic_info(cursor):
    queries = {
    "Total Suppliers":"SELECT COUNT(*) as total_suppliers FROM suppliers",
    
    "Total products": "SELECT count(*) as total_products FROM products",
    
    "Total category" :"SELECT count(distinct category) as total_category FROM products",
    
    "Total sales value made in last 3 months":"""SELECT ROUND(SUM(ABS(se.change_quantity) * p.price), 2) AS total_sales_last_3_months
    FROM stock_entries se
    JOIN products p 
    ON p.product_id = se.product_id
    WHERE se.change_type = 'Sale'
    AND se.entry_date >= (
        SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH)
        FROM stock_entries
         )""",
    
    
    "total restock value made in last 3 months":"""SELECT ROUND(SUM(ABS(se.change_quantity) * p.price), 2) AS total_restock_last_3_months
    FROM stock_entries se
    JOIN products p 
      ON p.product_id = se.product_id
    WHERE se.change_type = 'Restock'
      AND se.entry_date >= (
            SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH)
            FROM stock_entries
         )""",
     
"below reorder and no pending reorders":"""SELECT COUNT(*) FROM products as p
WHERE p.stock_quantity < p.reorder_level
AND p.product_id NOT IN (
SELECT DISTINCT product_id FROM reorders WHERE status = 'pending'
)"""	
}
result = {}
for label,query in queries.items():
    cursor.execute(query)
    row = cursor.fetchone()
        
    result[label] = list(row)[0]

In [17]:
def get_additional_tables(cursor):
    queries = {
        "Suppliers Contact details": "SELECT supplier_name,contact_name,email,phone FROM suppliers",

        "Product with supplier and stock": """SELECT p.product_name,s.supplier_name,p.stock_quantity,p.reorder_level 
                                            FROM products as p
                                            JOIN suppliers s ON
                                            p.supplier_id = s.supplier_id
                                            ORDER BY p.product_name ASC""",

        "Product needing reorder": """SELECT product_id,product_name,stock_quantity,reorder_level FROM products
                                    WHERE stock_quantity < reorder_level;"""
    }

    tables = {}
    for label, query in queries.items():
        cursor.execute(query)
        tables[label] = cursor.fetchall()

    return tables

In [18]:
def add_new_manual_id(cursor,db,p_name,p_category,p_price,p_stock,p_reorder,p_supplier):
    proc_call = "call AddNewProductManualId(%s,%s,%s,%s,%s,%s)"
    params = (p_name,p_category,p_price,p_stock,p_reorder,p_supplier)
    cursor.execute(proc_call,params)
    db.commit()

In [19]:
def get_categories(cursor):
    cursor.execute("SELECT DISTINCT category FROM products ORDER BY category ASC")
    rows = cursor.fetchall()
    return [row["category"]for row in rows]

In [20]:
def get_suppliers(cursor):
    cursor.execute("SELECT supplier_id , supplier_name FROM suppliers ORDER BY supplier_name ASC")
    return cursor.fetchall()

In [21]:
suppliers = get_suppliers(cursor)

In [22]:
supplier_ids=[s["Supplier_id"] for s in suppliers]
supplier_name=[s["Supplier_name"] for s in suppliers]

TypeError: tuple indices must be integers or slices, not str